In [1]:
import pandas as pd
import numpy as np
from future.utils.surrogateescape import encoded

from Models.DT import *
from utils.utils import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from Models.AutoEncoder import AutoEncoder, AE_trainDataset, AE_validDataset, AE_Dataset
from utils.utils import *
from tqdm import tqdm
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("Data/[24-2 DS_Project2] Data.csv")

In [3]:
data_path = 'Data/[24-2 DS_Project2] Data.csv'

In [4]:
cat_features = ['Gender', 'Card Brand', 'Card Type', 'Expires', 'Has Chip', 'Year PIN last Changed', 'Whether Security Chip is Used', 'Day', 'Error Message', 'WeekDay', 'Zipcode']

num_features = ['Current Age', 'Retirement Age', 'Per Capita Income - Zipcode', 'Yearly Income', 'Total Debt', 'Credit Score', 'Credit Limit', 'Amount','Since Open Month']

discarded = ['User', 'Birth Year', 'Birth Month', 'Card', 'Card Number', 'Merchandise Code', 'Acct Open Date', 'Year', 'Month']

In [5]:
(train_cat_X, train_num_X, train_y), (valid_cat_X, valid_num_X, valid_y), label_encoders = process_data(
    data_path,
    cat_features,
    num_features,
    discarded
)

TRANSITION
IQR
SPLIT
DISCARD
SCALE
ENCODE
UNLABEL
TARGET
TRAIN CAT/NUM
VALID CAT/NUM
RETURN


In [9]:
train_cat_X, 
train_num_X, 
train_y,
valid_cat_X,
valid_num_X,
valid_y,
label_encoders

{'Gender': LabelEncoder(),
 'Card Brand': LabelEncoder(),
 'Card Type': LabelEncoder(),
 'Expires': LabelEncoder(),
 'Has Chip': LabelEncoder(),
 'Year PIN last Changed': LabelEncoder(),
 'Whether Security Chip is Used': LabelEncoder(),
 'Day': LabelEncoder(),
 'Error Message': LabelEncoder(),
 'WeekDay': LabelEncoder(),
 'Zipcode': LabelEncoder()}

In [10]:
train_cat_X

,Gender,Card Brand,Card Type,Expires,Has Chip,Year PIN last Changed,Whether Security Chip is Used,Day,Error Message,WeekDay,Zipcode
1,0,2,1,57,1,6,0,26,0,0,0
2,0,2,1,57,1,6,1,26,0,0,0
3,0,2,1,57,1,6,0,27,0,2,0
5,0,2,1,57,1,6,0,4,0,2,0
6,0,2,1,57,1,6,0,8,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...
1642296,0,2,0,1,1,10,0,5,0,0,9
1642298,0,2,0,1,1,10,0,6,0,2,9
1642299,0,2,0,1,1,10,0,6,0,2,9
1642300,0,2,0,1,1,10,0,9,0,5,9


In [11]:
train_num_X

,Current Age,Retirement Age,Per Capita Income - Zipcode,Yearly Income,Total Debt,Credit Score,Credit Limit,Amount,Since Open Month
1,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,2.476193,1.335394
2,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,-0.729295,1.335394
3,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,1.763345,1.335394
5,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,0.207192,1.335394
6,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,-0.845773,1.335394
...,...,...,...,...,...,...,...,...,...
1642296,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.892365,-0.686084
1642298,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.789864,-0.686084
1642299,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.021106,-0.686084
1642300,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.850432,-0.686084


In [12]:
train_y

,Is Fraud?
1,0.0
2,0.0
3,0.0
5,0.0
6,0.0
...,...
1642296,0.0
1642298,0.0
1642299,0.0
1642300,0.0


In [13]:
valid_cat_X

,Gender,Card Brand,Card Type,Expires,Has Chip,Year PIN last Changed,Whether Security Chip is Used,Day,Error Message,WeekDay,Zipcode
172,0,2,1,57,1,6,0,25,0,4,0
173,0,2,1,57,1,6,0,26,0,0,0
174,0,2,1,57,1,6,0,27,0,2,0
175,0,2,1,57,1,6,0,27,0,2,0
176,0,2,1,57,1,6,0,28,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...
1642346,0,2,0,1,1,10,0,8,0,1,9
1642348,0,2,0,1,1,10,0,14,0,2,9
1642349,0,2,0,1,1,10,0,21,0,2,9
1642350,0,2,0,1,1,10,0,21,0,2,9


In [14]:
valid_num_X

,Current Age,Retirement Age,Per Capita Income - Zipcode,Yearly Income,Total Debt,Credit Score,Credit Limit,Amount,Since Open Month
172,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,-0.226108,1.505424
173,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,1.008564,1.505424
174,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,-0.710658,1.505424
175,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,-0.077015,1.505424
176,0.057673,-0.243686,1.156120,2.685432,2.270231,0.699536,1.237725,1.716753,1.505424
...,...,...,...,...,...,...,...,...,...
1642346,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.817818,-0.629407
1642348,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,-0.883046,-0.629407
1642349,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,0.952654,-0.629407
1642350,-0.319525,0.158174,-0.971574,-1.045608,0.144088,0.560190,-0.900764,0.053441,-0.629407


In [15]:
valid_y

,Is Fraud?
172,0.0
173,0.0
174,0.0
175,0.0
176,0.0
...,...
1642346,0.0
1642348,0.0
1642349,0.0
1642350,0.0


In [16]:
label_encoders

{'Gender': LabelEncoder(),
 'Card Brand': LabelEncoder(),
 'Card Type': LabelEncoder(),
 'Expires': LabelEncoder(),
 'Has Chip': LabelEncoder(),
 'Year PIN last Changed': LabelEncoder(),
 'Whether Security Chip is Used': LabelEncoder(),
 'Day': LabelEncoder(),
 'Error Message': LabelEncoder(),
 'WeekDay': LabelEncoder(),
 'Zipcode': LabelEncoder()}